In [1]:
import pandas as pd
import numpy as np
import pyodbc
import os
from pandas.tseries.offsets import MonthEnd, QuarterEnd

import matplotlib.pyplot as plt

In [2]:
# set working directory
os.chdir('../')
# print working directory
os.getcwd()

"c:\\Users\\XiaoA1\\OneDrive - Moody's\\COE - Credit Analytics-SQ - Unified ST Framework - Documents\\Sovereign Macro\\2 - Research Code"

In [3]:
version = '052025'

In [ ]:
# Part 1: Download raw data
# Connect to the database
edf9_connection = pyodbc.connect("DRIVER={SQL Server};SERVER=atco-datamart.prd.oaw.aws.moodys.tld;DATABASE=SIEDF_EDF9_CMA;UID=sqluser;PWD=readonly")
gcorr_connection = pyodbc.connect("DRIVER={SQL Server};SERVER=oac-wxresgco202;UID=sqluser;PWD=correlation1!")

# Query data from the databases
sov_cds_query = """
SELECT b.pid, b.entityName, b.cinc, b.mip_type, b.ndyNum, 
    a.red_code, a.price_date, a.cdsiedf1, a.cdsiedf2, a.cdsiedf3, a.cdsiedf4, a.cdsiedf5,
    a.spread1y, a.spread2y, a.spread3y, a.spread5y
FROM [SIEDF_EDF9].[dbo].[entity_cds_implied_edf] a
JOIN [CDS].[dbo].[siedf_map_master] b
ON a.red_code = b.mkmvcdsentityid
WHERE b.mip_type = 'Sov'
ORDER BY a.price_date
"""
sov_cds = pd.read_sql(sov_cds_query, edf9_connection)

sov_cma_query = """
SELECT b.pid, b.entityName, b.cinc, b.mip_type, b.ndyNum, 
    a.red_code, a.price_date, a.cdsiedf1, a.cdsiedf2, a.cdsiedf3, a.cdsiedf4, a.cdsiedf5,
    a.spread1y, a.spread2y, a.spread3y, a.spread5y
FROM [SIEDF_EDF9_CMA].[dbo].[entity_cds_implied_edf] a
JOIN [CDS].[dbo].[siedf_map_master] b
ON a.red_code = b.mkmvcdsentityid
WHERE b.mip_type = 'Sov'
ORDER BY a.price_date
"""
sov_cma = pd.read_sql(sov_cma_query, edf9_connection)

# Save raw data to CSV
sov_cds.to_csv('Output/1.cds_raw_'+version+'.csv', index=False)
sov_cma.to_csv('Output/1.cma_raw_'+version+'.csv', index=False)


C:\Users\XiaoA1\AppData\Local\Temp\1\ipykernel_12092\1668111258.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sov_cds = pd.read_sql(sov_cds_query, edf9_connection)
C:\Users\XiaoA1\AppData\Local\Temp\1\ipykernel_12092\1668111258.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sov_cma = pd.read_sql(sov_cma_query, edf9_connection)


In [ ]:

# Load data from CMA and Credit Edge datasets
cma = pd.read_csv('Output/1.cma_raw_'+version+'.csv')
cds = pd.read_csv('Output/1.cds_raw_'+version+'.csv')

cma['price_date'] = pd.to_datetime(cma['price_date'])
cds['price_date'] = pd.to_datetime(cds['price_date'])

# Create a flag indicating the data source
new = pd.merge(cds, cma, on=['pid', 'entityName', 'cinc', 'mip_type', 'ndyNum', 'red_code', 'price_date'], how='left')
diff = new[['cdsiedf1_y', 'cdsiedf2_y', 'cdsiedf3_y', 'cdsiedf4_y', 'cdsiedf5_y']].subtract(
    new[['cdsiedf1_x', 'cdsiedf2_x', 'cdsiedf3_x', 'cdsiedf4_x', 'cdsiedf5_x']].values
)
new['flag'] = diff.abs().sum(axis=1).apply(lambda x: 'CMA' if x < 1e-6 else 'Markit')
new.loc[new[['cdsiedf1_y', 'cdsiedf2_y', 'cdsiedf3_y', 'cdsiedf4_y', 'cdsiedf5_y']].isna().all(axis=1), 'flag'] = 'Markit'
new.loc[new['price_date'] <= '2015-07-14', 'flag'] = 'Markit' # add this logic to aviod removing old data by mistake

cds_data = pd.merge(cds, new[['pid', 'entityName', 'cinc', 'mip_type', 'ndyNum', 'red_code', 'price_date', 'flag']],
         on=['pid', 'entityName', 'cinc', 'mip_type', 'ndyNum', 'red_code', 'price_date'], how='left')

# Part 2: Combine 2 data sources
sov_data = cds_data[cds_data['mip_type'] == 'Sov']
sov_data['price_date'] = pd.to_datetime(sov_data['price_date'])
sov_data['yyyyqq'] = sov_data['price_date'] + QuarterEnd(0)
sov_data['yyyymm'] = sov_data['price_date'] + MonthEnd(0)
sov_data['week'] = sov_data['price_date'] - pd.to_timedelta(sov_data['price_date'].dt.weekday, unit='d')


sov_data = sov_data[~sov_data['pid'].isin(["S12801", "S10079", "S12945", "S10457", "S10607"])]

sov_markit = sov_data[sov_data['flag'] == 'Markit']
sov_cma = sov_data[sov_data['flag'] == 'CMA']

sovereigns = set(sov_markit['cinc']).intersection(set(sov_cma['cinc']))
sov_combined = pd.concat([sov_markit[sov_markit['cinc'].isin(sovereigns)],
                 sov_cma[sov_cma['cinc'].isin(sovereigns)]])
sov_combined.to_csv('Output/1.sov_combined_'+version+'.csv', index=False)

In [12]:
sov_markit['price_date'].max()

Timestamp('2015-07-14 00:00:00')

In [14]:
difference = set(sov_markit['cinc']).difference(set(sov_cma['cinc']))
print(difference)
difference = set(sov_cma['cinc']).difference(set(sov_markit['cinc']))
print(difference)

{'TWN', 'FJI', 'CIV'}
{'SAU'}


In [22]:
sov_data.loc[sov_data['pid'].isin(["S12801", "S10079", "S12945", "S10457", "S10607"]), ['entityName', 'pid']].drop_duplicates()

,entityName,pid
7,CENTRAL BANK OF TUNISIA,S10079


In [ ]:
# Part 3: Liquidity Filtering
regions = pd.read_csv("Input/country_list.csv")
quotes = pd.read_csv("Input/sov_quotes.csv")
sov_combined = pd.read_csv("Output/1.sov_combined_"+version+".csv")
sov_combined['price_date'] = pd.to_datetime(sov_combined['price_date'])
sov_combined['spread5y'] = pd.to_numeric(sov_combined['spread5y'], errors='coerce')
sov_combined['region'] = sov_combined['cinc'].map(regions.set_index('ISO')['region'])

# Filter 1: Number of observations
sov_nobs = sov_combined.groupby('cinc').agg(
    Nobs=('price_date', 'count'),
    Nweek=('week', 'nunique'),
    Nmonth=('yyyymm', 'nunique'),
    Nquarter=('yyyyqq', 'nunique'),
    start=('price_date', 'min'),
    end=('price_date', 'max'),
    region=('region', 'first')
).reset_index()

sov_nobs['Nobs_weight'] = sov_nobs['Nobs'] / sov_nobs['Nobs'].max()
sov_nobs['Nweek_weight'] = sov_nobs['Nweek'] / sov_nobs['Nweek'].max()

In [20]:
display(sov_nobs.sort_values(['Nweek_weight']).style.set_properties(**{'max-height': 'none', 'max-width': 'none'}))

,cinc,Nobs,Nweek,Nmonth,Nquarter,start,end,region,Nobs_weight,Nweek_weight
8,BLZ,2069,426,105,39,2009-12-09 00:00:00,2022-03-31 00:00:00,Latin America,0.409784,0.421365
23,ECU,2675,540,126,44,2006-01-06 00:00:00,2025-05-14 00:00:00,Latin America,0.529808,0.534125
54,MKD,2788,570,136,49,2011-09-20 00:00:00,2025-05-14 00:00:00,Southern Europe,0.552189,0.563798
84,VEN,3086,618,143,48,2006-01-06 00:00:00,2017-11-03 00:00:00,Latin America,0.611210,0.611276
35,HUN,3434,688,159,54,2012-03-16 00:00:00,2025-05-14 00:00:00,Eastern Europe,0.680135,0.680514
30,GHA,3486,702,164,57,2008-05-15 00:00:00,2023-02-27 00:00:00,Africa,0.690434,0.694362
37,IND,3520,708,165,57,2006-01-06 00:00:00,2025-05-14 00:00:00,Asia,0.697168,0.700297
72,SGP,3530,711,167,57,2006-02-08 00:00:00,2025-05-14 00:00:00,Asia,0.699148,0.703264
48,LBN,3686,738,170,57,2006-01-06 00:00:00,2020-02-21 00:00:00,Middle East,0.730046,0.729970
61,OMN,3728,755,177,60,2008-10-31 00:00:00,2025-05-14 00:00:00,Middle East,0.738364,0.746785


In [ ]:

sovereigns = sov_nobs[sov_nobs['Nweek_weight'] > 0.5]['cinc']

# Filter 2: Number of contributed quotes
quotes_use = quotes[quotes['Contributors_Mean'] > 2]
sovereigns_use = set(sovereigns).intersection(set(quotes_use['cinc']))

# Add additional sovereigns
index_composite = regions[regions['index_composite'] == True]['ISO']
additional_sovereigns = set(index_composite) - set(sovereigns_use)
sovereigns_use = sorted(sovereigns_use.union(additional_sovereigns))

# Save filtered data
sov_use = sov_combined[sov_combined['cinc'].isin(sovereigns_use)]
sov_use.to_csv('Output/1.sov_use_'+version+'.csv', index=False)

In [35]:
sov_month = sov_use.groupby(['pid', 'entityName', 'cinc', 'region', 'yyyymm', 'yyyyqq'])[['cdsiedf1', 'cdsiedf2', 'cdsiedf3', 'cdsiedf4', 'cdsiedf5']].mean().reset_index()
sov_month['yyyymm'] = pd.to_datetime(sov_month['yyyymm'])
sov_month

,pid,entityName,cinc,region,yyyymm,yyyyqq,cdsiedf1,cdsiedf2,cdsiedf3,cdsiedf4,cdsiedf5
0,S10023,"EGYPT, GOVERNMENT OF",EGY,Middle East,2006-01-31,2006-03-31,0.000100,0.000213,0.000491,0.000940,0.001612
1,S10023,"EGYPT, GOVERNMENT OF",EGY,Middle East,2006-02-28,2006-03-31,0.000100,0.000227,0.000527,0.001008,0.001722
2,S10023,"EGYPT, GOVERNMENT OF",EGY,Middle East,2006-03-31,2006-03-31,0.000100,0.000264,0.000609,0.001151,0.001941
3,S10023,"EGYPT, GOVERNMENT OF",EGY,Middle East,2006-04-30,2006-06-30,0.000114,0.000430,0.000948,0.001709,0.002762
4,S10023,"EGYPT, GOVERNMENT OF",EGY,Middle East,2006-05-31,2006-06-30,0.000126,0.000504,0.001095,0.001948,0.003107
...,...,...,...,...,...,...,...,...,...,...,...
15422,S13275,"Hungary, Government of",HUN,Eastern Europe,2025-01-31,2025-03-31,0.000536,0.001376,0.002402,0.003685,0.005254
15423,S13275,"Hungary, Government of",HUN,Eastern Europe,2025-02-28,2025-03-31,0.000499,0.001301,0.002293,0.003543,0.005080
15424,S13275,"Hungary, Government of",HUN,Eastern Europe,2025-03-31,2025-03-31,0.000443,0.001186,0.002121,0.003317,0.004801
15425,S13275,"Hungary, Government of",HUN,Eastern Europe,2025-04-30,2025-06-30,0.000425,0.001111,0.001983,0.003111,0.004528


In [52]:
sov_month_wide = sov_month.pivot(index='yyyymm', columns='cinc', values='cdsiedf5')
sov_month_wide.reset_index(inplace=True)
sov_month_wide

cinc,yyyymm,ARG,AUS,AUT,BEL,BGR,BHR,BRA,CAN,CHE,...,SVN,SWE,THA,TUN,TUR,UKR,USA,VEN,VNM,ZAF
0,2006-01-31,0.011873,0.000149,0.000151,0.000158,0.002310,0.001671,0.005259,0.000149,NaN,...,0.000240,0.000100,0.002233,0.002934,0.004361,0.005647,0.000100,0.005046,0.003742,0.000945
1,2006-02-28,0.009345,0.000171,0.000158,0.000163,0.002407,0.001662,0.003892,0.000159,NaN,...,0.000271,0.000125,0.002362,0.003006,0.004158,0.006290,0.000105,0.003920,0.003967,0.002521
2,2006-03-31,0.010851,0.000165,0.000179,0.000191,0.002420,0.001806,0.004308,0.000185,NaN,...,0.000281,0.000142,0.003165,0.003418,0.004530,0.006613,0.000118,0.004480,0.003694,0.002990
3,2006-04-30,0.013045,0.000169,0.000183,0.000207,0.002594,0.002012,0.005476,0.000203,NaN,...,0.000314,0.000138,0.003064,0.003872,0.005382,0.007029,0.000122,0.005347,0.003887,0.003409
4,2006-05-31,0.011853,0.000244,0.000173,0.000213,0.002825,0.002097,0.005500,0.000210,NaN,...,0.000338,0.000137,0.003042,0.004132,0.006832,0.007963,0.000124,0.005283,0.003366,0.001487
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,2025-01-31,0.064367,0.000320,0.000453,0.000789,0.003653,0.008189,0.008739,0.000668,0.000147,...,0.001170,0.000282,0.001503,0.091234,0.011506,NaN,0.001170,NaN,0.004277,0.009021
229,2025-02-28,0.067878,0.000316,0.000457,0.000774,0.003837,0.008217,0.008250,0.000679,0.000143,...,0.001222,0.000274,0.001454,0.077143,0.011336,NaN,0.001205,NaN,0.004256,0.009752
230,2025-03-31,0.064088,0.000335,0.000441,0.000764,0.003277,0.008382,0.008115,0.000645,0.000137,...,0.001204,0.000253,0.001531,0.078370,0.011967,NaN,0.001394,NaN,0.004023,0.009553
231,2025-04-30,0.051203,0.000316,0.000372,0.000675,0.002975,0.008013,0.006545,0.000539,0.000150,...,0.000973,0.000214,0.001744,0.069768,0.011692,NaN,0.001454,NaN,0.003856,0.009268


In [57]:
cinc_list = sov_month[['cinc', 'pid', 'entityName']].drop_duplicates()
for cinc, entityName in cinc_list[['cinc', 'entityName']].values:
    plt.figure(figsize=(10, 6))
    plt.title(cinc + ' ' + entityName)
    plt.plot(sov_month_wide['yyyymm'], sov_month_wide[cinc])
    plt.xlabel('Date')
    plt.ylabel('5Y CDS Implied EDF')
    plt.grid()
    plt.savefig(f'Plot/1.sov_use_monthly/{cinc}_5Y.png')
    plt.close()

In [58]:
sov_nobs_month = sov_month.groupby(['pid', 'entityName', 'cinc', 'region']).agg(
    Nmonth=('yyyymm', 'nunique'),
    Nquarter=('yyyyqq', 'nunique'),
    cdsiedf5_unique = ('cdsiedf5', 'nunique'),
    start=('yyyymm', 'min'),
    end=('yyyymm', 'max')
).reset_index()
sov_nobs_month['cdsiedf5_unique_weight'] = sov_nobs_month['cdsiedf5_unique'] / max(sov_nobs_month['Nmonth'])
display(sov_nobs_month.sort_values(['cdsiedf5_unique_weight']).style.set_properties(**{'max-height': 'none', 'max-width': 'none'}))
sov_nobs_month.to_csv('Output/1.sov_monthly_summary_'+version+'.csv', index=False)

,pid,entityName,cinc,region,Nmonth,Nquarter,cdsiedf5_unique,start,end,cdsiedf5_unique_weight
54,S10578,"VENEZUELA, GOVERNMENT OF",VEN,Latin America,143,48,143,2006-01-31 00:00:00,2017-11-30 00:00:00,0.613734
68,S13275,"Hungary, Government of",HUN,Eastern Europe,159,54,159,2012-03-31 00:00:00,2025-05-31 00:00:00,0.682403
34,S10551,"INDIA, GOVERNMENT OF",IND,Asia,165,57,165,2006-01-31 00:00:00,2025-05-31 00:00:00,0.708155
49,S10572,"SINGAPORE, GOVERNMENT OF",SGP,Asia,167,57,167,2006-02-28 00:00:00,2025-05-31 00:00:00,0.716738
22,S10410,"LEBANON, GOVERNMENT OF",LBN,Middle East,170,57,170,2006-01-31 00:00:00,2020-02-29 00:00:00,0.729614
61,S10659,"OMAN, GOVERNMENT OF",OMN,Middle East,177,60,177,2008-10-31 00:00:00,2025-05-31 00:00:00,0.759657
63,S10694,"UKRAINE, GOVERNMENT OF",UKR,Eastern Europe,192,65,192,2006-01-31 00:00:00,2022-06-30 00:00:00,0.824034
1,S10027,"ARGENTINA, GOVERNMENT OF",ARG,Latin America,208,72,193,2006-01-31 00:00:00,2025-05-31 00:00:00,0.828326
56,S10592,"RUSSIA, GOVERNMENT OF",RUS,Eastern Europe,198,66,198,2006-01-31 00:00:00,2022-06-30 00:00:00,0.849785
62,S10665,"SWITZERLAND, GOVERNMENT OF",CHE,Europe,199,68,198,2007-06-30 00:00:00,2025-05-31 00:00:00,0.849785
